In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time

class Model_Vgg19:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1_1 = tf.layers.conv2d(inputs=X_img, filters=64, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv1_2 = tf.layers.conv2d(inputs=conv1_1, filters=64, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1_2, pool_size=[2, 2], padding="SAME", strides=2)
            
            conv2_1 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=128, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2_2, pool_size=[2, 2], padding="SAME", strides=2)
            
            conv3_1 = tf.layers.conv2d(inputs=pool2, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv3_2 = tf.layers.conv2d(inputs=conv3_1, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv3_3 = tf.layers.conv2d(inputs=conv3_2, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv3_4 = tf.layers.conv2d(inputs=conv3_3, filters=256, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3_4, pool_size=[2, 2], padding="SAME", strides=2)
            '''
            conv4_1 = tf.layers.conv2d(inputs=pool3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv4_2 = tf.layers.conv2d(inputs=conv4_1, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv4_3 = tf.layers.conv2d(inputs=conv4_2, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv4_4 = tf.layers.conv2d(inputs=conv4_3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool4 = tf.layers.max_pooling2d(inputs=conv4_4, pool_size=[2, 2], padding="SAME", strides=2)
            
            conv5_1 = tf.layers.conv2d(inputs=pool4, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv5_2 = tf.layers.conv2d(inputs=conv5_1, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv5_3 = tf.layers.conv2d(inputs=conv5_2, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            conv5_4 = tf.layers.conv2d(inputs=conv5_3, filters=512, kernel_size=[3, 3], padding="SAME", activation=tf.nn.relu)
            pool5 = tf.layers.max_pooling2d(inputs=conv5_4, pool_size=[2, 2], padding="SAME", strides=2)
            '''            
            # Dense Layer with Relu
            flat6 = tf.reshape(pool3, [-1, 256 * 4 * 4])
            fc6 = tf.layers.dense(inputs=flat6, units=625, activation=tf.nn.relu)
            dropout6 = tf.layers.dropout(inputs=fc6, rate=0.5, training=self.training)
            '''
            flat7 = tf.reshape(dropout6, [-1, 4096])
            fc7 = tf.layers.dense(inputs=flat7, units=4096, activation=tf.nn.relu)
            dropout7 = tf.layers.dropout(inputs=fc7, rate=0.5, training=self.training)
            '''
            # Logits (no activation) Layer: L7 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout6, units=10)
            
        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.training: training})

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# feature 와 label 분리
train_y = train['label']
train_x = train.drop(labels='label', axis=1)

# label을 one-hot encoding
train_y_onehot = pd.get_dummies(train_y)

# train data count
total_train_rows = train_y_onehot.shape[0]

In [3]:
tf.set_random_seed(777)  # reproducibility

learning_rate = 0.001 
training_epochs = 20
batch_size = 100

# initialize
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) 

models = []
num_models = 2
for m in range(num_models):
    models.append(Model_Vgg19(sess, "model" + str(m)))
    
sess.run(tf.global_variables_initializer())

print('Learning Started!')

start_time = time.time()

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(total_train_rows / batch_size)
    start_time_epoch = time.time()
    for i in range(total_batch):
        begin = i * batch_size
        end = begin + batch_size
        batch_xs = train_x[begin : end].values
        batch_ys = train_y_onehot[begin : end].values

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list, " %.2f seconds" % (time.time() - start_time_epoch))

print('Learning Finished!')
print("--- %.2f seconds ---" %(time.time() - start_time))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Learning Started!


InvalidArgumentError: logits and labels must be same size: logits_size=[200,10] labels_size=[100,10]
	 [[Node: softmax_cross_entropy_with_logits_sg = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits_sg/Reshape, softmax_cross_entropy_with_logits_sg/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits_sg', defined at:
  File "/home/shkim/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shkim/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shkim/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shkim/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start